## Exercise 02.01: Simple Diagnostic Scores
* (Again) Read the paper "Martin Atzmueller, Joachim Baumeister, and Frank Puppe. Semi-Automatic Learning of Simple Diagnostic Scores utilizing Complexity Measures. Artificial Intelligence in Medicine, 37(1):19–30, 2006." (also discussed in the lecture): https://www.sciencedirect.com/science/article/pii/S0933365705000862
* Build on the implemention (you can of course your implementation) of the data structure diagnostic profile - relating to the description in the paper.
* From assignment 01: The class should have an (internal) method build which takes a dataframe, a list of diagnoses, and list of findings (in the form of the respective attributes) and constructs the according diagnostic profile object. You can, for example, implement this as a static method of the class, or use the respective instance method being called from the constructor of the class (for instantiating the object)
* From assignment 01: For this class, also implement a method prune(...) which takes an integer for removing infrequent findings
* Your class should contain a method learn(...) which learns the set of diagnostic scores (according to the method presented in the paper), respecting the given thresholds. Design the interface of your class/methods accordingly, e.g., such that the respective thresholds can be specified.
* For this you should create a new class DiagnosticScore which captures the simple diagnostic score pattern (simple finding -> solution relation); use this for representing diagnostic scores
* Test your implementation, via reading in and applying your implementation on this dataset: 
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer
* Here, you can consider the class attribute for the different diagnoses (i.e., consider the individual values of the class attribute as such).
* Implement a method printScores() which prints the learned scores (for debugging/diagnostic purposes) in tabular format.
* Also provide an according getter method for the learned set of diagnostic scores.
* For the diagnostic profile class, create a method "classify" which takes a row of a dataframe (e.g., from the test dataset above) and outputs the most likely class according to the obtained diagnostic score values. Here, implement the aggregation of scores in such a way, that, for example, for S1, S2, S3 - S1 accounts for half of S2, S2 accounts for half of S3, and that any a score larger than S2 (larger -> ">", not ">=" (!)) "establishes" the specific class
* Test your implementation - comparing the performance of the classification with the given class contained in the data, by printing out the "classified solution" and the "correct solution/class".

In [4]:
import math
import numpy as np
import pandas as pd
import re
from enum import Enum


class Score(Enum):
    S_MINUS_3 = "S_-3"
    S_MINUS_2 = "S_-2"
    S_MINUS_1 = "S_-1"
    S_0 = "S_0"
    S_1 = "S_1"
    S_2 = "S_2"
    S_3 = "S_3"


class DiagnosticScore:
    def __init__(self, diagnosis, finding, score: float):
        self.finding = finding
        self.diagnosis = diagnosis
        self.score = num_to_score(score)

    def __str__(self):
        return "Attribute:\t" + str(self.finding) + "Diagnosis:\t" + str(
            self.diagnosis) + "Diagnostic Score:\t" + self.score.value + "\n"

    def __repr__(self):
        return "\nAttribute:\t" + str(self.finding) + "\nDiagnosis:\t" + str(
            self.diagnosis) + "\nDiagnostic Score:\t" + self.score.value + "\n"


def score_to_num(score: Score) -> float:
    if score == Score.S_MINUS_3: return -1
    if score == Score.S_MINUS_2: return -0.5
    if score == Score.S_MINUS_1: return -0.25
    if score == Score.S_0: return 0
    if score == Score.S_1: return 0.25
    if score == Score.S_2: return 0.5
    if score == Score.S_3: return 1


def num_to_score(score: float) -> Score:
    if score < -0.9:
        return Score.S_MINUS_3
    if score < -0.5:
        return Score.S_MINUS_2
    if score < 0:
        return Score.S_MINUS_1
    if score < 0.5:
        return Score.S_1
    if score < 0.9:
        return Score.S_2
    else:
        return Score.S_3


class DiagnosticProfile:
    def __init__(self, df: pd.DataFrame, diagnosis: [], findings: []):

        # (<attribute value>, <frequency of attribute in conjunction with a diagnose)
        self.diagnostic_profiles = {}
        self.attributes = {}
        self.diagnosis = diagnosis
        self.rules = {}
        # self.row_to_attribute = {}
        column = 1
        # with regular expressions splice attributes and attribute values (easier to substitute the domain of application)
        for finding in findings:
            attribute = re.search(r'(.*):', finding).group(1)
            attribute = bytes(attribute, 'utf-8')
            values = re.search(r':\s(.*)', finding).group(1)
            values = values.split(', ')
            for i in range(len(values)):
                values[i] = bytes(values[i], 'utf-8')
            self.attributes[attribute] = values
            # self.row_to_attribute[column] = attribute
            column += 1
        df.columns = self.attributes.keys()
        self.attributes.pop(b'Class')
        self.case_base = df

        # initialize diagnostic_profiles structure
        for dia in diagnosis:
            self.diagnostic_profiles[dia] = {}
            for attribute in self.attributes.keys():
                self.diagnostic_profiles[dia][attribute] = {}

        # number of cases of every diagnosis
        self.num_cases = {}
        # zero cases at the start
        for diagnose in diagnosis:
            self.num_cases[diagnose] = 0

        for index, row in df.iterrows():
            dia = row[b'Class']
            self.num_cases[dia] += 1
            for attribute in self.attributes.keys():
                if row[attribute] in self.diagnostic_profiles[dia][attribute].keys():
                    self.diagnostic_profiles[dia][attribute][row[i]] += 1
                else:
                    self.diagnostic_profiles[dia][attribute][row[i]] = 1
        # normalize values to match frequencies
        for dia in self.diagnostic_profiles.keys():
            cases = self.num_cases[dia]
            for attr in self.diagnostic_profiles[dia]:
                for attr_val in self.diagnostic_profiles[dia][attr].keys():
                    self.diagnostic_profiles[dia][attr][attr_val] /= cases

    def prune(self, int_val: int):
        deleteList = np.empty((0, 3), bytes)
        # Delete all entries of too infrequent cases
        # For all diagnosis
        for dia in self.diagnostic_profiles.keys():
            # all findings
            for finding in self.diagnostic_profiles[dia].keys():
                # all attributevalues
                for attval in self.diagnostic_profiles[dia][finding].keys():
                    # check if threshold is met
                    if (self.diagnostic_profiles[dia][finding][attval] * self.num_cases[dia] < int_val):
                        # add to deletelist if below threshold
                        deleteEntry = np.array([dia, finding, attval], bytes).reshape(1, 3)
                        deleteList = np.append(deleteList, deleteEntry, axis=0)

        for entry in deleteList:
            del self.diagnostic_profiles[entry[0]][entry[1]][entry[2]]

    def learn(self, chi_alpha: float, threshold_c: float, threshold: int = 5, print_logs: bool = False):
        # 1: Für alle Diagnosen d aus dem Set der Diagnosen (z.B {Krebs, kein Krebs})
        for diagnosis in self.diagnosis:
            if print_logs: print("\nDiagnose:", diagnosis)
            # 2: Lerne ein Diagnoseprofil (bereits geschehen)
            # 3: Für alle Attribute a, für die ein Attributwert (z.B. alter 30-40) von a existiert
            #    und dieses in einem Diagnoseprofil der Diagnose d ist:
            for attribute in self.attributes.keys():
                if print_logs: print("|\tAttribute:", attribute)
                if not self.diagnostic_profiles[diagnosis][attribute]:
                    continue
                # 4: Für alle Attributewerte f des Attributs a:
                for attribute_value in self.attributes[attribute]:
                    if not (attribute_value in self.case_base[attribute].values):
                        continue
                    if print_logs: print("|\t|\tAttribut-Wert:", attribute_value)
                    # 5 Erstelle die binären Variablen D und F (für die Diagnose d und den Attributwert f)
                    a = b = c = d = 0
                    for index, row in self.case_base.iterrows():
                        if row[b'Class'] == diagnosis:
                            if row[attribute] == attribute_value:
                                a += 1
                            else:
                                c += 1
                        else:
                            if row[attribute] == attribute_value:
                                b += 1
                            else:
                                d += 1
                    # diagnosis and attribute value has to be co-ocorring at least <threshold> number of times
                    # otherwise the rule gets discarded
                    if a < threshold:
                        continue

                    # chi^2 test
                    chi_2 = self.compute_chi2(a, b, c, d)
                    if print_logs: print("|\t|\t|\tchi^2:", chi_2)
                    if chi_2 >= chi_alpha:
                        phi = self.compute_psi(a, b, c, d)
                        if print_logs: print("|\t|\t|\tphi", phi)
                        if print_logs: print("|\t|\t|\tqps", self.compute_qps(a, b, c, d))
                        if abs(phi) >= threshold_c:
                            if diagnosis not in self.rules.keys():
                                self.rules[diagnosis] = {}
                            self.rules[diagnosis][(attribute, attribute_value)] \
                                = DiagnosticScore(diagnosis, attribute_value, self.compute_qps(a, b, c, d))

    def classify(self, row: pd.Series):
        highscore = -math.inf
        output = b'default'
        # for all diagnosis comupte the diagnostioc score (initial highscore as low as possible)
        for diagnosis in self.diagnosis:
            score = 0
            for key in row.keys():
                if key == b'Class':
                    continue
                if (key, row[key]) in self.rules[diagnosis].keys():
                    score += score_to_num(self.rules[diagnosis][(key, row[key])].score)
            # set new highscore if diagnosis most likely till now
            if score > highscore:
                output = diagnosis
                highscore = score
        print("original: " + str(row[b'Class'], "utf-8") + "\t-\tclassified: " + str(output, "utf-8") + " with a diagnostic score of " + num_to_score(highscore).value)

    def get_diagnostic_score(self):
        return self.rules

    def print_scores(self) -> None:
        for diagnosis in self.rules.keys():
            print(str(diagnosis, "utf-8"))
            df = pd.DataFrame(list(self.rules[diagnosis]))
            df.columns = ["Attribute", "Attribute-Value"]
            scores = []
            for key in self.rules[diagnosis].keys():
                scores.append(self.rules[diagnosis][key].score.value)
            df.insert(2, "Score", scores)
            print(df)

    @staticmethod
    def compute_chi2(a: int, b: int, c: int, d: int) -> float:
        return ((a + b + c + d) * pow(a * d - b * c, 2)) / \
               ((a + b) * (c + d) * (a + c) * (b + d))

    @staticmethod
    def compute_psi(a: int, b: int, c: int, d: int) -> float:
        return (a * d - b * c) / \
               math.sqrt((a + b) * (c + d) * (a + c) * (b + d))

    @staticmethod
    def compute_precision(a: int, b: int, c: int, d: int) -> float:
        dependency = DiagnosticProfile.compute_chi2(a, b, c, d)
        if dependency > 0:
            return a / (a + b)
        elif dependency < 0:
            return b / (a + b)
        else:
            raise RuntimeWarning("Dependency should not be neutral")

    @staticmethod
    def compute_false_alarm_rate(a: int, b: int, c: int, d: int) -> float:
        dependency = DiagnosticProfile.compute_chi2(a, b, c, d)
        if dependency > 0:
            return b / (b + d)
        elif dependency < 0:
            return a / (a + c)
        else:
            raise RuntimeWarning("Dependency should not be neutral")

    @staticmethod
    def compute_qps(a: int, b: int, c: int, d: int) -> float:
        sign = lambda x: -1 if x < 0 else (1 if x > 0 else 0)
        return sign(DiagnosticProfile.compute_psi(a, b, c, d)) * \
               DiagnosticProfile.compute_precision(a, b, c, d) * \
               (1 - DiagnosticProfile.compute_false_alarm_rate(a, b, c, d))


if __name__ == '__main__':
    # example
    cases_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data'
    names_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.names'
    dafr = pd.DataFrame(np.loadtxt(cases_url, delimiter=",", dtype=bytes))
    dafr.columns = [b'class', b'age', b'menopause', b'tumor-size', b'inv-nodes', b'node-caps', b'deg-malig', b'breast',
                    b'breast-quad', b'irradiat']
    # copy from the names file... just need to remove the dots and replace the \n with "," for simplicity
    names = ['Class: no-recurrence-events, recurrence-events',
             'age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99',
             'menopause: lt40, ge40, premeno',
             'tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59',
             'inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39',
             'node-caps: yes, no',
             'deg-malig: 1, 2, 3',
             'breast: left, right',
             'breast-quad: left-up, left-low, right-up,	right-low, central',
             'irradiat:	yes, no']

    dp = DiagnosticProfile(dafr, [b'no-recurrence-events', b'recurrence-events'], names)
    dp.prune(0)
    dp.learn(0.1, 0.05, print_logs=False)
    dp.print_scores()
    for index, row in dafr.iterrows():
        dp.classify(row)


no-recurrence-events
         Attribute Attribute-Value Score
0           b'age'        b'30-39'  S_-1
1           b'age'        b'50-59'   S_2
2     b'menopause'         b'ge40'   S_1
3     b'menopause'      b'premeno'  S_-1
4    b'tumor-size'          b'0-4'   S_2
5    b'tumor-size'        b'10-14'   S_3
6    b'tumor-size'        b'30-34'  S_-1
7     b'inv-nodes'          b'0-2'   S_1
8     b'inv-nodes'          b'3-5'  S_-1
9     b'inv-nodes'          b'6-8'  S_-1
10    b'node-caps'          b'yes'  S_-1
11    b'node-caps'           b'no'   S_1
12    b'deg-malig'            b'1'   S_2
13    b'deg-malig'            b'2'   S_2
14    b'deg-malig'            b'3'  S_-1
15       b'breast'         b'left'  S_-1
16       b'breast'        b'right'   S_1
17  b'breast-quad'      b'central'   S_2
18     b'irradiat'          b'yes'  S_-1
19     b'irradiat'           b'no'   S_1
recurrence-events
        Attribute Attribute-Value Score
0          b'age'        b'30-39'   S_1
1          b'age'   

## Exercise 02.02: Reading/Discussion/Summary
* Read the paper "Neural-Symbolic Computing: An Effective Methodology for Principled Integration of Machine Learning and Reasoning", available here: https://arxiv.org/abs/1905.06088
* Think about the following questions:
** What is neuro-symbolic computing?
** Which problems does it solve?
** What are its advantages?
** Are there any disadvantages?
** What are specific challenges?
** What are some exemplary techniques to apply?
** How do they work?

* Prepare answers for these questions for the practical session on November 2, 2021. You will first discuss these in groups, and then we will discuss them in the plenary meeting.
* After that, summarize your findings (and those of the group discussion) in a small report (max. half a Din A4 page). For example, you could write 2-3 sentences for answering a specific question.

# Report 

Neural Cymbolic Computing is a strategy where Neural Networks are combined with symbolic representation of knowledge, in order to be able to both learn efficiently and robustly as well as reason from what has been learned. The symbolic knowledge makes it more understandable for humans and the neural networks can be used to infer on this knowledge.

Neural Symbolic Computing especially solves problem of explainability of AI systems using neural netorks. On top of that it offers techniques to prevent overfitting and increase learning efficiency when dealing with small amounts of learning data, as well as techniques for automatik knowledge extraction. 

Neural Symbolic Computing has the advantage of encompassing numerous techniques like the previously mentioned horizontal hybrid learning to improve learning with small datasets. On top of that, the increased understandability also makes it a lot esier to directly control and affect the learning process in intentional ways, like using previous knowledge encoded as parameters to improve learning. 

The disadvantages of Neural Symbolic Computing are not as numerous, but constructing a neural smybolic system instead of another AI system comes at the cost of needing to decide both for the symbolic knowledge representation as well as needing a sufficient amount of training data, making more effort necessary compare to some systems. On top of that the neural network part can give room to more errors compared to a well constructed purely symbolic system. 

Challenges of neural symbolic computation include challenges of specific techniques like propositionalisation generating irrelevant clauses. More generally, it is unclear what the best approach for constructing a neural smybolic system. Approximating satisfiability in neural symbolic reasoning is another challenge. 

An exemplary technique is horizontal hybrid learning, where prior symbolic knowledge is taken from networks or experts, encoded into a neural network, and used to improve the efficiency of learning and to prevent overfitting.  The prior knowledge is encoded as controlled parameters that directly affect learning. In knowledge transfer learning, the symbolic knowledge is extracted from one domain and then applied to learning of a neural symbolic system a related domain.

## Exercise 02.03: Description Logics
* Translate the example knowledge base given in KBS02 - slide 72 into first order predicate logic (see slide 71)
* Apply the axioms and rule of inference on the resulting expressions, and provide all possible inferences for the given vocabulary.

For providing this in the notebook, have a look here:
* https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html
* https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd

### Solution
#### RBox
$own \sqsubseteq careFor=\forall x\forall y ( own (x, y )\rightarrow careFor(x, y))$
#### TBox
$Healthy \sqsubseteq \lnot Dead = \forall x (Healthy(x)\rightarrow \lnot Dead(x)) \newline
Cat \sqsubseteq Dead \sqcup Alive = \forall x (Cat(x)\rightarrow Dead(x) \lor Alive(x))\newline
HappyCatOwner \sqsubseteq \exists owns.Cat \sqcap \forall caresFor.Healthy = \forall x (HappyCatOwner(x) \rightarrow \exists y(owns(x,y) \land Cat(y)) \land \forall y(caresFor(x,y) \rightarrow Healthy(y)))$
#### ABox
$HappyCatOwner(Schrödinger) = HappyCatOwner(Schrödinger)$


#### Results
Insert Fact HappyCatOwner(Schrödinger) in last TBox rule:
$HappyCatOwner(Schrödinger)\rightarrow \exists y(owns(Schrödinger,y) \land Cat(y)) \land \forall y(caresFor(Schrödinger,y) \rightarrow Healthy(y)))\\ $

insert for y "SchrödingersKatze":
$owns(Schrödinger,SchrödingersKatze)\newline
Cat(SchrödingersKatze))\\ $

Insert $owns(Schrödinger,SchrödingersKatze)$ in RBox:
$own (Schrödinger,SchrödingersKatze)\rightarrow careFor(Schrödinger,SchrödingersKatze)\\ $

Insert $careFor(Schrödinger,SchrödingersKatze)$ again in last TBox rule:
$caresFor(Schrödinger,SchrödingersKatze)\rightarrow Healthy(SchrödingersKatze)\\ $

Insert SchrödingesKatze for second rule in TBox:
$Cat(SchrödingersKatze)\rightarrow Dead(SchrödingersKatze) \lor Alive(SchrödingersKatze))\\ $

Insert $Healthy(SchrödingersKatze)$ for first rule in TBox:
$Healthy(SchrödingersKatze)\rightarrow \lnot Dead(SchrödingersKatze)\\ $

Inserting $\lnot Dead(SchrödingersKatze)$ in $ \forall x (Cat(x)\rightarrow Dead(x) \lor Alive(x))$ results in:
$Alive(SchrödingersKatze))$

## Exercise 02.04 Datalog

* Below, we consider the problem of finding (strongly) connected components in a directed graph.
For this, we can use the $edge(X, Y)$ predicate, such that this is true if there is an edge between nodes $X$ and $Y$. Let us assume we have a node and we want to find all the nodes that are in the same connected component as the given node. So, given a node $X$, find all those nodes $Y$ that are reachable from X and from which you can get back to X.

$inSCC(X,Y)\, :-  \,\, reachable(X,Z),\, reachable(Z,Y).$

$reachable(X,X).$

$reachable(X,Y)\, :-\,\, reachable(X,Z), edge(Z,Y).$

* Then, for a given node $X$, this will find all nodes in the same strongly connected component as $X$. However, this solution is quite inefficient, since it will in general take $O(n*e)$ time, where $n$ is the number of nodes and $e$ is the number of edges.

* However, we can provide a more efficient version, since this problem can be solved in $O(e)$ time. The core idea is, given a node $X$, find all nodes reachable from $X$ by following edges forward. Then, find all nodes reachable from $X$ by following edges backward (i.e., following edges against the arrow.) Finally, intersect those two sets. That will be the set of nodes in $X$'s strongly connected component, because if $Y$ is in both these sets, you can follow the edges forward from $X$ to $Y$ and then since there is also a backwards path from $X$ to $Y$, there is a forward path from $Y$ to $X$, so you can get from $X$ to $Y$ and back to $X$ following edges forward.

* Provide a Datalog implementation (using a set of rules) of this optimized algorithm.
* Provide a test case for your implementation, by specifying a small sample graph (let's say, with 10 nodes - where you create some edges there)
* Test your implementation and test case, e.g., using
** PyDatalog: https://sites.google.com/site/pydatalog/home
or
** AbcDatalog: https://abcdatalog.seas.harvard.edu/



In [1]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('X,Y,Z, inSCC, start,reachable_from_start, reverse_reachable_from_start, edge')
#rules
inSCC(X,Y) <= reachable_from_start(X,Y) & reverse_reachable_from_start(X,Y)
reachable_from_start(X,Y) <= reachable_from_start(X,Z) & edge(Z,Y) & start(X)
reverse_reachable_from_start(X,Y) <= reverse_reachable_from_start(X,Z) & edge(Y,Z) & start(X) 
#facts
start(X)
reachable_from_start(X,X)
reverse_reachable_from_start(X,X)

#network
+ edge('k1', 'k3')
+ edge('k1', 'k10')

+ edge('k2', 'k1')

+ edge('k3', 'k8')

+ edge('k4', 'k2')
+ edge('k4', 'k10')

+ edge('k5', 'k3')

#no k6 arc 

+ edge('k7', 'k4')
+ edge('k7', 'k9')

+ edge('k8', 'k5')

+ edge('k9', 'k4')
+ edge('k9', 'k7')

+ edge('k10', 'k1')
+ edge('k10', 'k4')
+ edge('k10', 'k6')


#given X
S = 'k4'

#input-dependent facts
+ start(S)
+ reachable_from_start(S,S)
+ reverse_reachable_from_start(S,S)

#result
print(inSCC(S, Y))

Y  
---
k10
k1 
k4 
k2 
